# Calculating SHAP values

## Importing objects and libraries

In [ ]:
#!pip install numba --upgrade

In [ ]:
#!pip install shap

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from scripts.preprocess import Transformer

In [ ]:
df = pd.read_csv('./data/data.csv', index_col = 0)
df.head()

,league,id,minute,X,Y,player,h_a,player_id,situation,season,shotType,match_id,home_team,away_team,home_goals,away_goals,date,player_assisted,lastAction,status
1,Ligue_1,425095,7,0.964,0.654,Myron Boadu,h,9612,OpenPlay,2021,LeftFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,NaN,BallRecovery,0
2,Ligue_1,425098,13,0.925,0.431,Gelson Martins,h,7012,OpenPlay,2021,RightFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,Caio Henrique,Throughball,1
3,Ligue_1,425100,24,0.785,0.388,Kevin Volland,h,83,OpenPlay,2021,LeftFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,NaN,None,0
4,Ligue_1,425101,24,0.761,0.525,Jean Lucas,h,7687,OpenPlay,2021,RightFoot,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,NaN,Rebound,0
5,Ligue_1,425102,30,0.936,0.415,Kevin Volland,h,83,FromCorner,2021,Head,17822,Monaco,Nantes,1,1,2021-08-06 19:00:00,Jean Lucas,Aerial,0


In [ ]:
tr = Transformer()
tr.fit(df.drop('status', axis = 1))

Transformer()

In [ ]:
import pickle

with open(r"./model/model.pickle", "rb") as output_file:
    model = pickle.load(output_file)

In [ ]:
import shap

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Save background predictions 

In [ ]:
X_oversampled = pd.read_csv('./data/dataset_train_oversampled.csv', index_col = 0)
y_hat_full = model.predict_proba(X_oversampled.drop('status', axis = 1))
pd.DataFrame({
    'y_hat' : y_hat_full[:,1]
}).to_csv('./model/y_hat.csv')

## SHAP explain object

In [ ]:
explainer = shap.TreeExplainer(model)

## Calculate SHAP values for Lewandowski's shots in 2021

In [ ]:
subset = df[np.logical_and(df.player == 'Robert Lewandowski', df.season == 2021)]

In [ ]:
y_subset = subset['status']
X_subset = subset.drop('status', axis = 1)

In [ ]:
X_subset = tr.transform(X_subset)

In [ ]:
shap_values = explainer.shap_values(X_subset)

In [ ]:
shap_values[1]

array([[-0.05886255, -0.00031832,  0.00575221, ..., -0.15994073,
        -0.08962472, -0.07927488],
       [-0.05906326, -0.01107966,  0.00266595, ..., -0.09795103,
        -0.10274493, -0.10511664],
       [-0.09893595, -0.04463718, -0.00074158, ..., -0.02931539,
        -0.00184785,  0.00797201],
       ...,
       [-0.07339138, -0.00738581, -0.00432754, ..., -0.01319115,
        -0.14489788, -0.26159488],
       [-0.08870353, -0.01701022, -0.00206603, ..., -0.02475384,
        -0.1800611 , -0.17425298],
       [-0.09420842, -0.0315397 ,  0.00204967, ..., -0.05303037,
        -0.1696245 , -0.12457114]])

In [ ]:
import pickle

with open(r"./shap_vals_example.pickle", "wb") as output_file:
    pickle.dump(shap_values, output_file)

In [ ]:
shaps = pd.DataFrame(shap_values[1])
shaps.columns =  X_subset.columns
shaps

,minute,h_a,situation,shotType,lastAction,distanceToGoal,angleToGoal
0,-0.058863,-0.000318,0.005752,-0.087726,-0.159941,-0.089625,-0.079275
1,-0.059063,-0.011080,0.002666,0.023294,-0.097951,-0.102745,-0.105117
2,-0.098936,-0.044637,-0.000742,0.017511,-0.029315,-0.001848,0.007972
3,-0.101141,-0.002665,-0.129321,-0.004385,-0.098672,-0.079724,-0.044087
4,-0.131909,-0.027483,-0.021399,-0.093407,-0.061691,-0.062777,-0.061328
...,...,...,...,...,...,...,...
155,-0.107138,-0.035834,0.001149,0.004059,-0.055519,-0.146461,-0.140252
156,-0.098747,-0.014584,-0.002025,-0.089257,-0.161179,-0.065593,-0.038611
157,-0.073391,-0.007386,-0.004328,0.004793,-0.013191,-0.144898,-0.261595
158,-0.088704,-0.017010,-0.002066,-0.003148,-0.024754,-0.180061,-0.174253


In [ ]:
shaps.to_csv('./treeshap_shaps.csv')

In [ ]:
y_hat = model.predict_proba(X_subset)
pd.DataFrame({
    'y_hat' : y_hat[:,1]
}).to_csv('./y_hat_subset.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3965177e-3b24-40f1-9445-2feb56564f30' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>